#### Rough Plan

- Provide recommendatins of other house plants to buy based on the ones you have/like.
- Can also add a suprise me method, which shows most disimilar. 
- Figure out a data source with features needed, then extract and store locally. 
- Content-Based Recommendation will be used.
- Can make a dash app to host it

### Option 1:

- Tropical plant database, misses quite a lot but really nicely detailed so easy to extract data from.

- iterate through these web pages and a make a pandas table with each piece of info.

- Does miss out on several classic types of house plants though... 

http://www.tropicopia.com/house-plant/detail.np/detail-01.html

https://www.tropicopia.com/house-plant/detail.np/detail-355.html

#### Option 2:

- Much more options here, still categorical info so easy enough to handle.
- Challenge is there is clear overlap for some of the plants so not sure how best to handle that. 
- Could try to filter by creating a list of items to search through using a website (shop) or other database.

http://www.missouribotanicalgarden.org/PlantFinder/PlantFinderListResults.aspx?letter=A

## Going with option 3:

- Use a plant shop website to extract a list of house plants or multiple websites.
- Probably most clever as identifies those that are actually available.
- Then maybe database search each of these entries to extract info for each? 
- Will try from https://www.plantagen.se/vaxter-inomhus/ Need swedish name and latin name. 


TODO - running into issues with uniquesness associated with the plantagen and latin names.

https://www.houseplant.co.uk/collections/house-plants-indoor-plants?page=3 - issues here too with non plant named stuff


Potentially another way to think about this problem is take only the latin names and search for matches of those?
So kind of ignore the Swedish name component entirely... and take the common names. Maybe this makes more sense... 


In [1]:
import re
import requests
from typing import Tuple
from bs4 import BeautifulSoup

In [69]:
# def get_blomsterlandet_plants(url:str) -> dict:
#     """
#     Using the website: https://www.blomsterlandet.se to identify all the houseplants they have available.
    
#     Works nicely! :) 
#     TODO - fill this part in...
    
#     """
#     r = requests.get(url)
#     soup = BeautifulSoup(r.content, 'lxml')


#     swedish_section = soup.find_all('h2', class_="ProductCardBodystyled__Name-lrzi29-2 PFKhc")
#     science_section = soup.find_all('p', class_="ProductCardBodystyled__ScientificName-lrzi29-3 flSCLr")

#     plant_names = {}
#     for swedish_name, science_name in zip(swedish_section, science_section):
#         # Using the swedish_name instead of science_name  important.
#         # Some diff plants have same science_name and this auto deals with duplicates
#         # duplicates occur for different size pot etc... but same plant... 
#         plant_names.update({swedish_name.get_text(): science_name.get_text()})

#     return {key: value for key, value in sorted(plant_names.items())}


def get_blomsterlandet_plants(url:str) -> list:
    """
    Using the website: https://www.blomsterlandet.se to identify all the houseplants they have available.
    
    Works nicely! :) 
    TODO - fill this part in...
    
    """
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'lxml')

    science_section = soup.find_all('p', class_="ProductCardBodystyled__ScientificName-lrzi29-3 flSCLr")

    # plant_names = []
    # for science_name in science_section:
    #     # Some diff plants have same science_name and this auto deals with duplicates
    #     # duplicates occur for different size pot etc... but same plant... 
    #     plant_names.append(science_name.get_text())

    plant_names = [science_name.get_text() for science_name in science_section] 

    return plant_names



In [ ]:
blomsterlandet_url = "https://www.blomsterlandet.se/produkter/vaxter/inomhus/grona-vaxter/?page=50&sorting=Name&filterDefaults=false"
plant_names = get_blomsterlandet_plants(url=blomsterlandet_url)

In [73]:
len(plant_names)
#len(set(plant_names))

640

In [24]:
for swedish_name, science_name in plant_names.items():
    print(swedish_name + ": " + science_name)

Agave-echeveria: Echeveria agavoides
Alocasia 'Pink Passion': Alocasia brancifolia
Alocasia 'Silver Dragon': Alocasia baginda
Aloe 'Black Gem': Aloe
Aloegasteria 'Cosmo': x Gasteraloe
Alokasia: Alocasia macrorrhizos
Alokasia 'Black Velvet': Alocasia
Alokasia 'Macrophylla': Alocasia
Alokasia 'Pink Dragon': Alocasia
Alokasia 'Portodora': Alocasia
Alokasia 'Sarian': Alocasia
Alokasia 'Stingray': Alocasia macrorrhizos
Alokasia 'Yucatan Princess': Alocasia macrorrhizos
Amazonsköld 'Polly': Alocasia x amazonica
Amazonsköld BAMBINO ARROW: Alocasia x amazonica
Amazonsköld CURLY BAMBINO: Alocasia x amazonica
Amerikansk hällebräken: Woodsia obtusa
Ampellilja 'Bonnie', grönbladig: Chlorophytum comosum
Ampellilja 'Irish': Chlorophytum comosum
Ampellilja 'Mandarin': Chlorophytum comosum
Ampellilja 'Variegatum': Chlorophytum comosum
Ampellilja Ampel: Chlorophytum comosum
Antennbräken: Doryopteris pilosa
Anthurium 'Jungle King': Anthurium crassinervium
Arabiskt kaffe: Coffea arabica
Aralia: Fatsia ja

In [36]:
# Next step is to generate a function that does all the filtering I need. 

# Then identify those that have a unique science_name can be dealt with easily.

# Those without filter to seperate place and deal with....

In [50]:
# Initial clean attempt.
all_science_names = list(plant_names.values())
unique_plants = [swedish_name for swedish_name in plant_names if all_science_names.count(plant_names[swedish_name]) == 1]
#non_unique_plants = [swedish_name for swedish_name in plant_names if all_science_names.count(plant_names[swedish_name]) != 1]

print(len(unique_plants), len(non_unique_plants))

185 347


In [54]:
unique_plant_dict, non_unique_plant_dict = {}, {}
for swedish_name, science_name in plant_names.items():
    if swedish_name in unique_plants:
        unique_plant_dict.update({swedish_name: science_name})
    else: 
        non_unique_plant_dict.update({swedish_name: science_name})

Will press ahead with the unique_plant_dict and work on extending to non_unique_plants later...
central idea for that fix up might be including the 'word' in the search term from the swedish name but that is not 100% either. 

In [68]:
example_subset = list(unique_plant_dict.values())[0:5]
example_subset

['Echeveria agavoides',
 'Alocasia brancifolia',
 'Alocasia baginda',
 'Aloe',
 'x Gasteraloe']

In [67]:
url_start = "https://google.com/search?q="
url_end = "%20site:http://www.missouribotanicalgarden.org"
url = url_start + "%20" + example_subset[1] + "%20" + url_end # + "%20" 
url

'https://google.com/search?q=%20Alocasia brancifolia%20%20site:http://www.missouribotanicalgarden.org'

In [ ]:
def search_for_plant(plant_dict: dict):
    """
    Generate a google query and run the query to extract the top results, html link
    for plants availble from the website: https://www.missouribotanicalgarden.org/PlantFinder/
    """
    params = {"gl": "us", "hl": "en"}
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.79 Safari/537.36",
    }
    url_start = "https://google.com/search?q="
    url_end = "%20site:http://www.missouribotanicalgarden.org"


    for swedish_name, science_name in unique_plant_dict.items():
        # this needs some thought as has to be variable by the number of words... 
        url = url_start + "%20" + "stuff to go here" + "%20" + url_end # + "%20" 


        request_result=requests.get(url, params=params, headers=headers, timeout=30)
        soup = BeautifulSoup(request_result.text, "html.parser")

        match_me = "https://www.missouribotanicalgarden.org/PlantFinder/PlantFinderDetails"

        if link!= None and match_me in link:
            print(link)
        the_one = link
        break
            






In [ ]:
# Now need to search the scientific name and extract the url of top results from this website:
# https://www.missouribotanicalgarden.org/PlantFinder/
# https://cse.google.com/cse?cx=015816930756675652018:7gxyi5crvvu&q=Test%20name
# Need to build the url first.

# Note, will need several improvments here, like variable number of names,
# or deal with fact some are in string quotes etc... 
# Also might need to check for duplicates etc...

compare_url = "https://cse.google.com/cse?cx=015816930756675652018:7gxyi5crvvu&q=Test%20name"
test_plant = science_name[2].get_text()

url_pre = "https://cse.google.com/cse?cx=015816930756675652018:7gxyi5crvvu&q="

first_name = test_plant.split(" ")[0]
last_name = test_plant.split(" ")[1]

url = url_pre + first_name + "%20" + last_name

url == compare_url
url

In [ ]:
r2 = requests.get(url)
soup2 = BeautifulSoup(r2.content, 'lxml')

In [ ]:
print(soup2)

In [ ]:
# # https://docs.python-requests.org/en/master/user/quickstart/#passing-parameters-in-urls
params = {
#"q": "Core Banking Solution Accenture",  # search query
"gl": "us",                              # country of the search
"hl": "en"                               # language                
}

# https://docs.python-requests.org/en/master/user/quickstart/#custom-headers
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.79 Safari/537.36",
}

text= "geeksforgeeks"
site_limit = "%20site:http://www.missouribotanicalgarden.org"
url = "https://google.com/search?q=" + first_name + "%20" + last_name + site_limit
  
# Fetch the URL data using requests.get(url),
# store it in a variable, request_result.
request_result=requests.get(url, params=params, headers=headers, timeout=30)# 
  
# Creating soup from the fetched request
soup = BeautifulSoup(request_result.text,"html.parser")

url

In [ ]:
url

In [ ]:
test = soup.find_all('a')

In [ ]:
links = []
for link in soup.find_all('a'):
    links.append(link.get('href'))
links[0]

In [ ]:
match_me = "https://www.missouribotanicalgarden.org/PlantFinder/PlantFinderDetails"

for link in links:
    if link!= None and match_me in link:
        print(link)
        the_one = link
        break
    #else:
       # print("not_found")

In [ ]:
# Final part, now I have the web address I want, I need to scrape from this webpage, all the details I want
r3 = requests.get(the_one)
soup3 = BeautifulSoup(r3.content, 'lxml')

In [ ]:
soup3.find_all("div", class_="row")

In [ ]:
# Need to take the 2nd element to the end to match... 
soup3.find("div", {"id": "MainContentPlaceHolder_TypeRow"}).contents[0].strip()

In [ ]:
#These below are easy to take. 
# MainContentPlaceHolder_CommonNameRow
# "MainContentPlaceHolder_FamilyRow"
# "MainContentPlaceHolder_NativeRangeRow"
# "MainContentPlaceHolder_ZoneRow"
# "MainContentPlaceHolder_HeightRow"
# "MainContentPlaceHolder_SpreadRow"
# "MainContentPlaceHolder_BloomTimeRow"
# "MainContentPlaceHolder_ColorTextRow"
# "MainContentPlaceHolder_SunRow"
# "MainContentPlaceHolder_WaterRow"
# "MainContentPlaceHolder_MaintenanceRow"

# These not so much....
# "row"Flower: Showy, Fragrant</div
# "row"Leaf: Evergreen</div
# "row"Fruit: Showy, Edible</div

In [ ]:

soup3.find(string=re.compile("Flower: "))
soup3.find(string=re.compile("Leaf: "))
soup3.find(string=re.compile("Fruit: "))